# PM 5 Task H Eigenmode Truncation


In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import eigs
import time
import ipywidgets as widgets

# Import sonar simulation functions
from getParam_Sonar import getParam_Sonar
from eval_f_Sonar import eval_f_Sonar
from eval_g_Sonar import eval_g_Sonar
from eval_u_Sonar import eval_u_Sonar_20, eval_u_Sonar_20_const
from simpleLeapFrog import LeapfrogSolver
from sonar_viz import plot_pressure_xz_at

In [ ]:
FAST_TEST = True  # Set to False for full PM5 run

print(f"Mode: {'FAST TEST (20 Hz)' if FAST_TEST else 'FULL RUN (3000 Hz)'}")

## Full Order Model

In [ ]:
if FAST_TEST:
    # 20 Hz in coarse domain 
    Lx, Lz = 4e3, 1e3      
    Nx, Nz = 161, 41  
    f0 = 20               
    eval_u = eval_u_Sonar_20_const      
else:
    Lx, Lz = 8e3, 1e3      
    Nx, Nz = 301, 51  
    f0 = 20               
    eval_u = eval_u_Sonar_20_const  

N = Nx * Nz
n = 2 * N

print(f"Domain: {Lx}m x {Lz}m")
print(f"Grid: {Nx} x {Nz} = {N:,} spatial points")
print(f"State dimension: 2N = {n:,}")

# Get parameters (this builds A and B)
p, x_start, t_start, t_stop, max_dt_FE = getParam_Sonar(
    Nx, Nz, Lx, Lz, UseSparseMatrices=True, BC=True
)

t_sim = t_stop 

# Extract system matrices
A = p['A']
B = p['B']

print(f"\nGrid spacing: dx = {p['dx']:.4f}m, dz = {p['dz']:.4f}m")
print(f"Sound speed: c = {p['c']} m/s")
print(f"Max stable dt: {max_dt_FE:.2e}s")

# Wavelength check
wavelength = p['c'] / f0
print(f"\nAt f₀ = {f0} Hz:")
print(f"  Wavelength: λ = {wavelength:.2f}m")
print(f"  Points per wavelength: {wavelength/p['dx']:.0f}")
print(f"  Domain coverage: {Lx/wavelength:.1f}λ × {Lz/wavelength:.1f}λ")


### Input function

In [ ]:
# Scale by cell area (same as PM1)
def eval_u_scaled(t):
    return (p['dx'] * p['dz']) * eval_u(t)

# Visualize pulse
t_test = np.linspace(0, t_sim, 1000)   
u_test = [eval_u(t) for t in t_test]

plt.figure(figsize=(10, 3))
plt.plot(t_test * 1000, u_test, 'b-', linewidth=1.5)
plt.xlabel('Time (ms)')
plt.ylabel('u(t)')
plt.title(f'Input Pulse: {f0} Hz Gaussian-windowed sine')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# FFT of pulse
t_test = np.linspace(0, t_sim, 10000)
u_test = np.array([eval_u_scaled(t) for t in t_test])

from scipy.fft import fft, fftfreq
U_fft = np.abs(fft(u_test))
freqs_fft = fftfreq(len(t_test), t_test[1] - t_test[0])

plt.figure(figsize=(10, 4))
plt.plot(freqs_fft[:len(freqs_fft)//2], U_fft[:len(U_fft)//2])
plt.axvline(x=f0, color='r', linestyle='--', label=f'f₀={f0} Hz')
plt.xlabel('Frequency (Hz)')
plt.ylabel('|U(f)|')
plt.title('Pulse Frequency Content')
plt.xlim(0, 2*f0)
plt.legend()
plt.grid(True)
plt.show()

# Where is the energy?
peak_freq = freqs_fft[np.argmax(U_fft[:len(U_fft)//2])]
print(f"Peak frequency: {peak_freq:.1f} Hz")

### Output matrix

In [ ]:
# Use hydrophones from getParam_Sonar
n_phones = p['hydrophones']['n_phones']
h_z = p['hydrophones']['z_pos']
h_x_indices = p['hydrophones']['x_indices']

# C matrix selects pressure at hydrophone locations
C = np.zeros((n_phones, n))
for i, hx in enumerate(h_x_indices):
    idx = hx * Nz + h_z
    C[i, N + idx] = 1.0

print(f"Hydrophones: {n_phones} receivers")
for i, hx in enumerate(h_x_indices):
    print(f"  H{i+1}: x = {hx * p['dx']:.0f}m")

## Compute all eigenmodes
Uniform damping so real part is the same for all modes (each mode decays at the same rate)

Save ahead of time...

In [ ]:
'''
t0 = time.perf_counter()

if FAST_TEST:
    # SMALL GRID: Dense eigensolve (all modes)
    print("Computing ALL eigenvalues (dense)...")
    A_dense = A.toarray()
    
    eigenvalues, V = np.linalg.eig(A_dense)
    k = len(eigenvalues)
    
    # Solve V @ b_tilde = B instead of b_tilde = V_inv @ B
    B_dense = B.toarray() if sp.issparse(B) else np.array(B)
    b_tilde = np.linalg.solve(V, B_dense)  # More stable than inv(V) @ B
    
    c_tilde = C @ V  # This is fine as-is
    
    print(f"✓ Complete: found all {k} eigenvalues")

else:
    # LARGE GRID: Sparse eigensolve near pulse frequency

    k_target = 5000  # Number of modes to compute
    sigma = 1j * 2 * np.pi * f0  # Shift near pulse frequency
    
    print(f"Computing {k_target} eigenvalues near f₀={f0} Hz (sparse)...")
    
    # Right eigenvectors
    eigenvalues, V = eigs(A, k=k_target, sigma=sigma, which='LM')
    
    # Left eigenvectors
    eigenvalues_L, W = eigs(A.T.conj(), k=k_target, sigma=np.conj(sigma), which='LM')
    
    # Match left to right by eigenvalue
    W_matched = np.zeros_like(W)
    for i in range(len(eigenvalues)):
        distances = np.abs(eigenvalues_L - eigenvalues[i])
        j = np.argmin(distances)
        W_matched[:, i] = W[:, j]
    W = W_matched
    
    # Normalize for biorthogonality
    for i in range(len(eigenvalues)):
        scale = np.conj(W[:, i]) @ V[:, i]
        if np.abs(scale) > 1e-10:
            W[:, i] = W[:, i] / np.conj(scale)
    
    k = len(eigenvalues)
    
    # Compute b_tilde and c_tilde using left eigenvectors
    B_dense = B.toarray() if sp.issparse(B) else np.array(B)
    
    b_tilde = np.conj(W.T) @ B_dense  # EXACT projection with left eigenvectors
    c_tilde = C @ V
    
    freqs = np.abs(eigenvalues.imag) / (2 * np.pi)
    print(f"✓ Complete: found {k} eigenvalues")
    print(f"  Frequency range: [{np.min(freqs):.1f}, {np.max(freqs):.1f}] Hz")

eig_time = time.perf_counter() - t0

# COMMON ANALYSIS
print(f"\nEigensolve time: {eig_time:.1f}s")
print(f"\nEigenvalue statistics:")
print(f"  Real parts: [{np.min(eigenvalues.real):.2e}, {np.max(eigenvalues.real):.2e}]")
print(f"  Frequencies: [{np.min(np.abs(eigenvalues.imag))/(2*np.pi):.1f}, {np.max(np.abs(eigenvalues.imag))/(2*np.pi):.1f}] Hz")

n_unstable = np.sum(eigenvalues.real > 1e-6)
print(f"  Stability: {'✓ All stable' if n_unstable == 0 else f'⚠ {n_unstable} unstable modes'}")

print(f"\nProjection matrices:")
print(f"  b_tilde: {b_tilde.shape}, |b̃| range: [{np.min(np.abs(b_tilde)):.2e}, {np.max(np.abs(b_tilde)):.2e}]")
print(f"  c_tilde: {c_tilde.shape}, |c̃| range: [{np.min(np.abs(c_tilde)):.2e}, {np.max(np.abs(c_tilde)):.2e}]")

save_dict = {
    'eigenvalues': eigenvalues,
    'V': V,
    'b_tilde': b_tilde,
    'c_tilde': c_tilde,
    'Nx': Nx,
    'Nz': Nz,
    'Lx': Lx,
    'Lz': Lz,
    'k': k,
    't0': t0,
}

np.savez('eigen_decomp.npz', **save_dict)
print("Saved to eigen_decomp.npz")
'''

Load pre computed eigenvalues

In [ ]:
data = np.load('eigen_decomp.npz', allow_pickle=True)

eigenvalues = data['eigenvalues']
V = data['V']
k = data['k']

'''
# these are for source at z index 1
b_tilde = data['b_tilde']
c_tilde = data['c_tilde']
Nx = int(data['Nx'])
Nz = int(data['Nz'])
Lx = float(data['Lx'])
Lz = float(data['Lz'])
t0 = float(data['t0'])
'''

B_dense = B.toarray() if sp.issparse(B) else np.array(B)
b_tilde = np.linalg.solve(V, B_dense)  # More stable than inv(V) @ B

c_tilde = C @ V  # This is fine as-is

print(f"Loaded: {len(eigenvalues)} eigenvalues, V shape {V.shape}")
print(f"Grid: {Nx} x {Nz}, Domain: {Lx} x {Lz}")

In [ ]:
# Plot eigenvalue spectrum
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
freqs = eigenvalues.imag / (2 * np.pi)
ax1.scatter(eigenvalues.real, freqs, c=np.abs(eigenvalues), cmap='viridis', s=50, alpha=0.7)
ax1.axvline(x=0, color='r', linestyle='--', linewidth=1, label='Stability boundary')
ax1.axhline(y=f0, color='g', linestyle=':', linewidth=2, alpha=0.7, label=f'Target f₀={f0} Hz')
ax1.axhline(y=-f0, color='g', linestyle=':', linewidth=2, alpha=0.7)
ax1.set_xlabel('Re(λ)')
ax1.set_ylabel('Im(λ)/(2π) [Hz]')
ax1.set_title('Eigenvalue Spectrum')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.hist(np.abs(freqs), bins=30, edgecolor='black', alpha=0.7)
ax2.ticklabel_format(useOffset=False)
ax2.axvline(x=f0, color='r', linestyle='--', linewidth=2, label=f'Target f₀={f0} Hz')
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel('Count')
ax2.set_title('Eigenmode Frequency Distribution')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("\nComputing I/O coupling scores...")

# Scores already use b_tilde and c_tilde computed above
scores = np.abs(b_tilde.flatten()) * np.linalg.norm(c_tilde, axis=0)

# Sort by score
idx_sorted = np.argsort(scores)[::-1]
scores_sorted = scores[idx_sorted]

cumsum = np.cumsum(scores_sorted)
total = cumsum[-1]
energy_pct = 0.90  # Capture __% of total coupling
q_energy = np.searchsorted(cumsum, energy_pct * total) + 1
print(f"\nCapture {100*energy_pct:.0f}% of total score:")
print(f"  q = {q_energy}")

log_scores = np.log10(scores_sorted + 1e-50)  # Avoid log(0)
diffs = np.diff(log_scores)  # Negative values = drops

# First significant drop (threshold in decades)
drop_threshold = 0.5  # adjust as needed
q_drop = np.argmax(-diffs > drop_threshold) + 1
if -diffs[q_drop - 1] <= drop_threshold:
    # No drop exceeds threshold, fall back to largest
    q_drop = np.argmin(diffs) + 1
drop_magnitude = -diffs[q_drop - 1]
print(f"\nFirst drop > {drop_threshold} decades:")
print(f"  q = {q_drop}, drop = {drop_magnitude:.1f} decades")

fig, ax = plt.subplots(figsize=(10, 5))
ax.semilogy(scores_sorted, '.-', markersize=2)
ax.axvline(x=q_energy, color='g', linestyle='--', label=f'90% energy: q={q_energy}')
ax.axvline(x=q_drop, color='orange', linestyle='--', label=f'First drop > {drop_threshold}: q={q_drop}')
ax.set_xlabel('Mode rank')
ax.set_ylabel('I/O coupling score')
ax.set_title('Score dropoff with automatic q selection')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Build ROM

Select modes based on how strongly they couple input to output  -->  Score_i = ||b̃_i|| × ||c̃_i||

Then build ROM

- Aq = diag(λ_selected) — diagonal matrix with eigenvalues
- Bq = b̃[selected]
- Cq = c̃[:, selected]
- Vq = V[:, selected] — projection for state reconstruction


In [ ]:
q = q_drop  # q_drop or q_energy
print(f"\nBuilding ROM with q = {q} modes...")

#q = 3500

In [ ]:
# Select top q modes by I/O coupling
idx_selected = idx_sorted[:q]

# Extract components
lambda_q = eigenvalues[idx_selected]
V_q = V[:, idx_selected]           # (n × q) projection matrix

A_q = np.diag(lambda_q)            # (q × q) diagonal!
B_q = b_tilde[idx_selected, :]     # (q × 1)
C_q = c_tilde[:, idx_selected]     # (n_phones × q)

print(f"\n✓ ROM built:")
print(f"  Full dimension:    n = {n:,}")
print(f"  Reduced dimension: q = {q}")
print(f"  Compression:       {100*q/n:.2f}%")
print(f"\n  Aq: {A_q.shape} (diagonal)")
print(f"  Bq: {B_q.shape}")
print(f"  Cq: {C_q.shape}")
print(f"  Vq: {V_q.shape}")

# Stability guaranteed!
print(f"\n✓ Stability: max(Re(λ)) = {np.max(lambda_q.real):.2e}")

Visualize selected frequencies

In [ ]:
# All eigenvalue frequencies
freqs_all = np.abs(eigenvalues.imag) / (2 * np.pi)

# Selected eigenvalue frequencies
freqs_selected = np.abs(lambda_q.imag) / (2 * np.pi)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Histogram comparison
ax1 = axes[0]
ax1.hist(freqs_all, bins=50, alpha=0.5, label=f'All modes (k={k})', color='gray', edgecolor='black')
ax1.hist(freqs_selected, bins=50, alpha=0.7, label=f'Selected (q={q})', color='tab:blue', edgecolor='black')
ax1.axvline(x=f0, color='r', linestyle='--', linewidth=2, label=f'Pulse f₀={f0} Hz')
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Count')
ax1.set_title('Frequency Distribution: All vs Selected Modes')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Scatter showing selection by I/O coupling score
ax2 = axes[1]
scores_all = np.abs(b_tilde.flatten()) * np.linalg.norm(c_tilde, axis=0)

# Not selected (gray)
mask_not_selected = np.ones(k, dtype=bool)
mask_not_selected[idx_selected] = False
ax2.scatter(freqs_all[mask_not_selected], scores_all[mask_not_selected], 
            s=20, alpha=0.3, color='gray', label='Not selected')

# Selected (colored by rank)
ax2.scatter(freqs_selected, scores_all[idx_selected], 
            c=np.arange(q), s=50, cmap='viridis', edgecolors='black', linewidths=0.5, label='Selected')

ax2.axvline(x=f0, color='r', linestyle='--', linewidth=2, label=f'Pulse f₀={f0} Hz')
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('I/O Coupling Score')
ax2.set_yscale('log')
ax2.set_title('Mode Selection by I/O Coupling')
ax2.legend()
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

# Summary stats
print(f"\nSelected frequency statistics:")
print(f"  Min: {np.min(freqs_selected):.1f} Hz")
print(f"  Max: {np.max(freqs_selected):.1f} Hz")
print(f"  Mean: {np.mean(freqs_selected):.1f} Hz")
print(f"  Median: {np.median(freqs_selected):.1f} Hz")
print(f"  Modes within ±10 Hz of pulse: {np.sum(np.abs(freqs_selected - f0) <= 10)}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.scatter(freqs, scores, s=20, alpha=0.5, label='All modes')
#ax.scatter(freqs[idx_sorted[:20]], scores[idx_sorted[:20]], 
           #s=100, c='red', edgecolors='black', label='Top 20')

ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('I/O Coupling Score')
ax.set_yscale('log')
ax.set_title('Dominant Eigenmodes by Frequency')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Check how much energy each mode has at surface (should be ~0 for good modes)
def check_surface_energy(V, Nx, Nz):
    N = Nx * Nz
    n_modes = V.shape[1]
    
    surface_energy = np.zeros(n_modes)
    for m in range(n_modes):
        mode = V[:, m]
        # Surface nodes: velocity at k=i*Nz (j=0), pressure at N+k
        for i in range(Nx):
            k = i * Nz
            surface_energy[m] += np.abs(mode[k])**2      # velocity
            surface_energy[m] += np.abs(mode[N + k])**2  # pressure
    
    return surface_energy

surface_energy = check_surface_energy(V, Nx, Nz)
print(f"Max surface energy per mode: {surface_energy.max():.2e}")
print(f"Modes with significant surface energy: {np.sum(surface_energy > 1e-2)}")

## Simulate ROM

Since Aq is diagonal, each mode is an independent scalar ODE:
# 
dx̃_i/dt = λ_i · x̃_i + Bq[i] · u(t)

In [ ]:
def simulate_rom_leapfrog_with_bc(A_q, B_q, C_q, V_q, u_func, t_span, dt, Nx, Nz, x0=None):
    """
    Leapfrog ROM with BC enforcement at each step.
    """
    q = A_q.shape[0]
    n_out = C_q.shape[0]
    N = Nx * Nz
    
    t = np.arange(t_span[0], t_span[1] + dt, dt)
    n_steps = len(t)
    
    x_q = np.zeros((n_steps, q), dtype=complex)
    y = np.zeros((n_steps, n_out), dtype=complex)
    
    lam = np.diag(A_q)
    B_flat = B_q.flatten()
    
    # Precompute BC-enforced V_q (zero out surface rows)
    V_q_bc = V_q.copy()
    for i in range(Nx):
        k = i * Nz
        V_q_bc[k, :] = 0        # velocity at surface
        V_q_bc[N + k, :] = 0    # pressure at surface
    
    # Modified C that outputs with BC enforcement
    # C_bc = C_q uses original C, but we reconstruct with V_q_bc
    
    if x0 is not None:
        x_prev = x0.flatten().astype(complex)
        x_curr = x0.flatten().astype(complex)
    else:
        x_prev = np.zeros(q, dtype=complex)
        x_curr = np.zeros(q, dtype=complex)
    
    # Bootstrap with RK4
    u0 = u_func(t[0])
    k1 = lam * x_curr + B_flat * u0
    k2 = lam * (x_curr + 0.5*dt*k1) + B_flat * u_func(t[0] + 0.5*dt)
    k3 = lam * (x_curr + 0.5*dt*k2) + B_flat * u_func(t[0] + 0.5*dt)
    k4 = lam * (x_curr + dt*k3) + B_flat * u_func(t[0] + dt)
    x_next = x_curr + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)
    
    x_q[0] = x_curr
    y[0] = C_q @ x_curr
    x_prev = x_curr.copy()
    x_curr = x_next.copy()
    
    # Leapfrog loop
    for i in range(1, n_steps):
        x_q[i] = x_curr
        y[i] = C_q @ x_curr
        if i < n_steps - 1:
            u_i = u_func(t[i])
            f_i = lam * x_curr + B_flat * u_i
            x_next = x_prev + 2 * dt * f_i
            x_prev = x_curr.copy()
            x_curr = x_next.copy()
    
    return t, np.real(y), x_q


# After simulation, reconstruct with BC
def reconstruct_with_bc(x_rom, V_selected, Nx, Nz):
    N = Nx * Nz
    X = (V_selected @ x_rom.T).real
    for i in range(Nx):
        k = i * Nz
        X[k, :] = 0
        X[N + k, :] = 0
    return X


def simulate_rom_leapfrog(A_q, B_q, C_q, u_func, t_span, dt, x0=None):
    """
    Leapfrog for diagonal ROM - O(q) per step, not O(q²).
    """
    q = A_q.shape[0]
    n_out = C_q.shape[0]
    
    t = np.arange(t_span[0], t_span[1] + dt, dt)
    n_steps = len(t)
    
    x_q = np.zeros((n_steps, q), dtype=complex)
    y = np.zeros((n_steps, n_out), dtype=complex)
    
    lam = np.diag(A_q)
    B_flat = B_q.flatten()
    
    # Initial condition
    if x0 is not None:
        x_prev = x0.flatten().astype(complex)
        x_curr = x0.flatten().astype(complex)
    else:
        x_prev = np.zeros(q, dtype=complex)
        x_curr = np.zeros(q, dtype=complex)
    
    # Bootstrap with RK4
    u0 = u_func(t[0])
    k1 = lam * x_curr + B_flat * u0
    k2 = lam * (x_curr + 0.5*dt*k1) + B_flat * u_func(t[0] + 0.5*dt)
    k3 = lam * (x_curr + 0.5*dt*k2) + B_flat * u_func(t[0] + 0.5*dt)
    k4 = lam * (x_curr + dt*k3) + B_flat * u_func(t[0] + dt)
    x_next = x_curr + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)
    
    x_q[0] = x_curr
    y[0] = C_q @ x_curr
    x_prev = x_curr.copy()
    x_curr = x_next.copy()
    
    # Leapfrog loop
    for i in range(1, n_steps):
        x_q[i] = x_curr
        y[i] = C_q @ x_curr
        if i < n_steps - 1:
            u_i = u_func(t[i])
            f_i = lam * x_curr + B_flat * u_i
            x_next = x_prev + 2 * dt * f_i
            x_prev = x_curr.copy()
            x_curr = x_next.copy()
    
    return t, np.real(y), x_q

In [ ]:
# Choose integrator: 'exact' or 'leapfrog' or 'bc'
ROM_INTEGRATOR = 'bc'

# Timestep 
dt = max_dt_FE * 0.1

print(f"Running ROM simulation (q={q}, integrator={ROM_INTEGRATOR})...")
print(f"  Duration: {t_sim*1000:.0f} ms")
print(f"  Timestep: {dt*1e6:.1f} μs")
print(f"  Steps: {int(t_sim/dt):,}")

# Project full-order x_start to eigenspace
x0_full = x_start.flatten()

if FAST_TEST:
    x0_eigen_all = np.linalg.solve(V, x0_full)
else:
    x0_eigen_all = np.conj(W.T) @ x0_full

x0_eigen_q = x0_eigen_all[idx_selected]

V_selected = V[:, idx_selected]

t0 = time.perf_counter()

if ROM_INTEGRATOR == 'bc':
    t_rom, y_rom, x_rom = simulate_rom_leapfrog_with_bc(
        A_q, B_q, C_q, V_selected, eval_u_scaled, (0, t_sim), dt, Nx, Nz, x0=x0_eigen_q
    )
else:
    t_rom, y_rom, x_rom = simulate_rom_leapfrog(A_q, B_q, C_q, eval_u_scaled, (0, t_sim), dt, x0=x0_eigen_q)

rom_time = time.perf_counter() - t0

print(f"\n✓ ROM complete in {rom_time:.3f}s")

X_rom_reconstructed = reconstruct_with_bc(x_rom, V_selected, Nx, Nz)

# Verify BC enforcement
N = Nx * Nz
surface_p = X_rom_reconstructed[N::Nz, :]
print(f"Surface pressure max: {np.abs(surface_p).max():.2e} (should be ~0)")

# Plot ROM hydrophone responses
fig, axes = plt.subplots(n_phones, 1, figsize=(12, 2.5*n_phones), sharex=True)
if n_phones == 1:
    axes = [axes]

for i in range(n_phones):
    axes[i].plot(t_rom*1000, y_rom[:, i], 'b-', linewidth=1)
    axes[i].set_ylabel('Pressure (Pa)')
    axes[i].set_title(f'ROM: Hydrophone {i+1} (x = {h_x_indices[i]*p["dx"]:.0f}m)')
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.show()

## Full Order Reference
Run the full model with LeapfrogSolver for comparison

In [ ]:
# Always run in FAST_TEST mode; optional in full mode
RUN_FULL_ORDER = True  # Set to False to skip in full mode

if RUN_FULL_ORDER:
    print(f"Running full-order simulation (n={n:,})...")
    if not FAST_TEST:
        print("⚠ This will take a while...")
    
    num_steps = int(np.ceil(t_sim / dt))
    
    t0 = time.perf_counter()
    X_ref, t_ref = LeapfrogSolver(
        eval_f_Sonar, 
        x_start, 
        p, 
        eval_u_scaled, 
        num_steps, 
        dt, 
        visualize=False,
        verbose=True
    )
    full_time = time.perf_counter() - t0
    
    # Extract hydrophone outputs using eval_g_Sonar
    print("Extracting hydrophone data...")
    y_ref = np.zeros((n_phones, len(t_ref)))
    for i in range(len(t_ref)):
        y_ref[:, i] = eval_g_Sonar(X_ref[:, i], p).flatten()
    
    print(f"\n✓ Full-order complete in {full_time:.1f}s")
    print(f"  SPEEDUP: {full_time/rom_time:.0f}x faster with ROM!")
else:
    t_ref, y_ref, full_time = None, None, None
    print("Skipping full-order simulation")

## Compare

In [ ]:
if y_ref is not None:
    # Interpolate ROM to match reference time points
    from scipy.interpolate import interp1d
    
    errors = []
    
    fig, axes = plt.subplots(n_phones, 2, figsize=(14, 3*n_phones))
    if n_phones == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(n_phones):
        # Interpolate ROM output
        y_rom_interp = np.interp(t_ref, t_rom, y_rom[:, i])
        
        # Error
        err = y_ref[i, :] - y_rom_interp
        rel_err = np.linalg.norm(err) / (np.linalg.norm(y_ref[i, :]) + 1e-12)
        errors.append(rel_err)
        
        # Left: overlay
        axes[i, 0].plot(t_ref*1000, y_ref[i, :], 'b-', linewidth=1.5, label='Full', alpha=0.7)
        axes[i, 0].plot(t_rom*1000, y_rom[:, i], 'r--', linewidth=1.5, label=f'ROM (q={q})')
        axes[i, 0].set_ylabel('Pressure (Pa)')
        axes[i, 0].set_title(f'H{i+1}: Full vs ROM')
        axes[i, 0].legend()
        axes[i, 0].grid(True, alpha=0.3)
        
        # Right: error
        axes[i, 1].plot(t_ref*1000, err, 'g-', linewidth=1)
        axes[i, 1].set_ylabel('Error (Pa)')
        axes[i, 1].set_title(f'H{i+1} Error (rel: {100*rel_err:.2f}%)')
        axes[i, 1].grid(True, alpha=0.3)
    
    axes[-1, 0].set_xlabel('Time (ms)')
    axes[-1, 1].set_xlabel('Time (ms)')
    plt.tight_layout()
    plt.show()
    
    # Summary
    print("=" * 60)
    print("EIGENMODE ROM RESULTS")
    print("=" * 60)
    print(f"Full-order dimension:  n = {n:,}")
    print(f"Reduced dimension:     q = {q}")
    print(f"Compression:           {100*q/n:.4f}%")
    print(f"\nAccuracy (relative L2 error):")
    for i, err in enumerate(errors):
        print(f"  Hydrophone {i+1}: {100*err:.2f}%")
    print(f"\nTiming:")
    print(f"  Full-order: {full_time:.1f}s")
    print(f"  ROM:        {rom_time:.3f}s")
    print(f"  Speedup:    {full_time/rom_time:.0f}x")
    print("=" * 60)

Visualize... do the BCs work?

In [ ]:
# Reconstruct full state from ROM eigenspace
# x_rom shape: (n_steps, q) → need to transpose
V_selected = V[:, idx_selected]
X_rom_reconstructed = (V_selected @ x_rom.T).real  # (2N, n_steps)

print(f"X_rom_reconstructed shape: {X_rom_reconstructed.shape}")
print(f"X_full shape: {X_ref.shape}")

# Visualize ROM only
slider = widgets.IntSlider(
    value=0, 
    min=0, 
    max=X_rom_reconstructed.shape[1] - 1, 
    step=1, 
    description='Index'
)
ui = widgets.HBox([slider])
out = widgets.interactive_output(
    lambda Index: plot_pressure_xz_at(p, X_rom_reconstructed, t_rom, index=Index), 
    {'Index': slider}
)
display(ui, out)

In [ ]:
def plot_full_vs_rom(p, X_full, X_rom, t_full, t_rom, index=0):
    """Plot full-order and ROM pressure fields side by side"""
    Nx, Nz = p['Nx'], p['Nz']
    N = Nx * Nz
    
    # Match time index (in case different dt)
    idx_full = min(index, X_full.shape[1] - 1)
    idx_rom = min(index, X_rom.shape[1] - 1)
    
    # Extract pressure (second half of state)
    p_full = X_full[N:2*N, idx_full].real.reshape(Nx, Nz).T
    p_rom = X_rom[N:2*N, idx_rom].real.reshape(Nx, Nz).T
    
    # Common color scale
    vmax = max(np.abs(p_full).max(), np.abs(p_rom).max())
    if vmax < 1e-12:
        vmax = 1e-12
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 4))
    
    # Full-order
    im0 = axes[0].imshow(p_full, extent=[0, p['Lx'], p['Lz'], 0],
                          cmap='RdBu_r', vmin=-vmax, vmax=vmax, aspect='auto')
    axes[0].set_xlabel('X (m)')
    axes[0].set_ylabel('Z (m)')
    axes[0].set_title(f'Full-order (t={t_full[idx_full]*1000:.1f} ms)')
    plt.colorbar(im0, ax=axes[0], label='Pressure (Pa)')
    
    # ROM
    im1 = axes[1].imshow(p_rom, extent=[0, p['Lx'], p['Lz'], 0],
                          cmap='RdBu_r', vmin=-vmax, vmax=vmax, aspect='auto')
    axes[1].set_xlabel('X (m)')
    axes[1].set_ylabel('Z (m)')
    axes[1].set_title(f'ROM q={X_rom.shape[0]//2} (t={t_rom[idx_rom]*1000:.1f} ms)')
    plt.colorbar(im1, ax=axes[1], label='Pressure (Pa)')
    
    # Error
    error = p_rom - p_full
    err_max = np.abs(error).max()
    if err_max < 1e-15:
        err_max = 1e-15
    im2 = axes[2].imshow(error, extent=[0, p['Lx'], p['Lz'], 0],
                          cmap='RdBu_r', vmin=-err_max, vmax=err_max, aspect='auto')
    axes[2].set_xlabel('X (m)')
    axes[2].set_ylabel('Z (m)')
    rel_err = np.linalg.norm(error) / (np.linalg.norm(p_full) + 1e-12) * 100
    axes[2].set_title(f'Error (rel: {rel_err:.2e}%)')
    plt.colorbar(im2, ax=axes[2], label='Error (Pa)')
    
    plt.tight_layout()
    plt.show()

# Reconstruct
V_selected = V[:, idx_selected]
X_rom_reconstructed = (V_selected @ x_rom.T).real

# Match lengths
n_times = min(X_ref.shape[1], X_rom_reconstructed.shape[1])

slider = widgets.IntSlider(
    value=0, 
    min=0, 
    max=n_times - 1, 
    step=10,  # step by 10 for faster scrubbing
    description='Index'
)
ui = widgets.HBox([slider])
out = widgets.interactive_output(
    lambda Index: plot_full_vs_rom(p, X_ref[:, :n_times], X_rom_reconstructed[:, :n_times], 
                                    t_ref[:n_times], t_rom[:n_times], index=Index), 
    {'Index': slider}
)
display(ui, out)

## Convergence study 
Vary q

In [ ]:
if y_ref is not None:
    q_values = [100, 500, 1000, 1500, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
    q_values = q_values + [q_energy, q_drop] 
    q_values = sorted(list(set(q_values)))  # Unique and sorted
    q_values = [qv for qv in q_values if qv <= k]  # Can't exceed computed modes
    
    print("Convergence study...")
    results = []
    
    for q_test in q_values:
        idx_sel = idx_sorted[:q_test]
        
        A_test = np.diag(eigenvalues[idx_sel])
        B_test = b_tilde[idx_sel, :]
        C_test = c_tilde[:, idx_sel]
        
        t0_s = time.perf_counter()
        _, y_test, _ = simulate_rom_leapfrog(A_test, B_test, C_test, eval_u_scaled, (0, t_sim), dt)
        sim_t = time.perf_counter() - t0_s
        
        # Error on first hydrophone
        y_interp = np.interp(t_ref, t_rom, y_test[:, 0])
        rel_err = np.linalg.norm(y_ref[0, :] - y_interp) / (np.linalg.norm(y_ref[0, :]) + 1e-12)
        
        speedup = full_time / sim_t if sim_t > 0 else 0
        
        results.append({'q': q_test, 'error': rel_err, 'time': sim_t, 'speedup': speedup})
        print(f"  q={q_test:4d}: error={100*rel_err:6.2f}%, time={sim_t:.4f}s, speedup={speedup:.1f}x")
    
    # Extract data
    qs = [r['q'] for r in results]
    errs = [100*r['error'] for r in results]
    speedups = [r['speedup'] for r in results]
    
    # Find indices for special points
    idx_energy = qs.index(q_energy) if q_energy in qs else None
    idx_drop = qs.index(q_drop) if q_drop in qs else None
    
    # ================================================================
    # PLOT 1: Error and Speedup vs q (dual y-axes)
    # ================================================================
    fig, ax1 = plt.subplots(figsize=(10, 6))
    
    # Left axis: Error (log scale)
    color1 = 'tab:blue'
    ax1.set_xlabel('Reduced Order q')
    ax1.set_ylabel('Relative Error (%)', color=color1)
    ax1.semilogy(qs, errs, 'o-', color=color1, linewidth=2, markersize=10, label='Error')
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.axhline(y=10, color='r', linestyle='--', alpha=1, label='10% error')
    ax1.axhline(y=1, color='g', linestyle='--', alpha=1, label='1% error')
    
    # Highlight q_drop (green star) and q_energy (orange square) on error axis
    if idx_drop is not None:
        ax1.scatter([qs[idx_drop]], [errs[idx_drop]], marker='*', s=300, 
                    color='lime', edgecolors='black', linewidths=1.5, zorder=5, 
                    label=f'Largest drop: q={q_drop}')
    if idx_energy is not None:
        ax1.scatter([qs[idx_energy]], [errs[idx_energy]], marker='s', s=150, 
                    color='orange', edgecolors='black', linewidths=1.5, zorder=5, 
                    label=f'90% energy: q={q_energy}')
    
    # Right axis: Speedup (inverted)
    ax2 = ax1.twinx()
    color2 = 'tab:orange'
    ax2.set_ylabel('Speedup (x)', color=color2)
    ax2.plot(qs, speedups, 'o--', color=color2, linewidth=2, markersize=8, label='Speedup')
    ax2.tick_params(axis='y', labelcolor=color2)
    ax2.invert_yaxis()
    
    # Highlight q_drop (green star) and q_energy (orange square) on speedup axis
    if idx_drop is not None:
        ax2.scatter([qs[idx_drop]], [speedups[idx_drop]], marker='*', s=300, 
                    color='lime', edgecolors='black', linewidths=1.5, zorder=5)
    if idx_energy is not None:
        ax2.scatter([qs[idx_energy]], [speedups[idx_energy]], marker='s', s=150, 
                    color='orange', edgecolors='black', linewidths=1.5, zorder=5)
    
    # Combined legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='center right')
    
    ax1.set_title('Eigenmode ROM: Error vs Speedup Tradeoff')
    ax1.grid(True, alpha=0.3, which='both')
    
    plt.tight_layout()
    plt.show()
    
    # ================================================================
    # PLOT 2: Error vs Speedup (Pareto)
    # ================================================================
    fig, ax = plt.subplots(figsize=(8, 6))
    
    ax.semilogy(speedups, errs, 'o-', linewidth=2, markersize=12, color='tab:blue')
    
    # Label each point with q value
    for i, q_val in enumerate(qs):
        ax.annotate(f'q={q_val}', (speedups[i], errs[i]), 
                    textcoords='offset points', xytext=(5, 5), fontsize=9)
    
    # Highlight q_drop (green star) and q_energy (orange square)
    if idx_drop is not None:
        ax.scatter([speedups[idx_drop]], [errs[idx_drop]], marker='*', s=400, 
                   color='lime', edgecolors='black', linewidths=2, zorder=5, 
                   label=f'Largest drop: q={q_drop}')
    if idx_energy is not None:
        ax.scatter([speedups[idx_energy]], [errs[idx_energy]], marker='s', s=200, 
                   color='orange', edgecolors='black', linewidths=2, zorder=5, 
                   label=f'90% energy: q={q_energy}')
    
    # Reference lines
    ax.axhline(y=10, color='r', linestyle='--', alpha=0.5, label='10% error')
    ax.axhline(y=1, color='g', linestyle='--', alpha=0.5, label='1% error')
    
    ax.set_xlabel('Speedup (x)')
    ax.set_ylabel('Relative Error (%)')
    ax.set_title('Eigenmode ROM: Accuracy vs Speedup Tradeoff')
    ax.legend()
    ax.grid(True, alpha=0.3, which='both')
    
    plt.tight_layout()
    plt.show()

print("\n" + "="*60)
print("EIGENMODE TRUNCATION MOR - COMPLETE")
print("="*60)
print(f"Method: Dominant poles with I/O coupling selection")
print(f"Key advantage: Diagonal Aq preserves stability")
print("="*60)

In [ ]:
'''
# RECONFIGURE FOR VERTICAL HYDROPHONE ARRAY

# Vertical array at center x
p['hydrophones'] = {
    'x_pos': p['sonar_ix'],
    'z_indices': list(range(1, Nz-1)),
    'n_phones': Nz - 2
}

# Rebuild C matrix for vertical array
n_phones = p['hydrophones']['n_phones']
C = np.zeros((n_phones, 2*N))
for i, zi in enumerate(p['hydrophones']['z_indices']):
    flat_idx = p['hydrophones']['x_pos'] * Nz + zi
    C[i, N + flat_idx] = 1.0

# Recompute c_tilde with new C
c_tilde = C @ V

# Recompute mode selection with new c_tilde
scores = np.abs(b_tilde.flatten()) * np.linalg.norm(c_tilde, axis=0)
idx_sorted = np.argsort(scores)[::-1]
idx_selected = idx_sorted[:q]

# Rebuild ROM matrices
A_q = np.diag(eigenvalues[idx_selected])
B_q = b_tilde[idx_selected]
C_q = c_tilde[:, idx_selected]

print(f"Reconfigured: {n_phones} vertical hydrophones at x={p['sonar_ix']*p['dx']:.0f}m")

# VERTICAL ARRAY COMPARISON

# Reconstruct ROM state
V_selected = V[:, idx_selected]
X_rom_reconstructed = (V_selected @ x_rom.T).real

# Extract vertical arrays
def extract_vertical_array(p, X, x_pos=None):
    Nx, Nz = p['Nx'], p['Nz']
    N = Nx * Nz
    if x_pos is None:
        x_pos = Nx // 2
    z_indices = list(range(1, Nz - 1))
    pressure = np.zeros((len(z_indices), X.shape[1]))
    for di, zi in enumerate(z_indices):
        flat_idx = x_pos * Nz + zi
        pressure[di, :] = X[N + flat_idx, :]
    depths = np.array(z_indices) * p['dz']
    return pressure, depths

pressure_full, depths = extract_vertical_array(p, X_ref)
pressure_rom, _ = extract_vertical_array(p, X_rom_reconstructed)

# Match lengths
n_times = min(pressure_full.shape[1], pressure_rom.shape[1])
pressure_full = pressure_full[:, :n_times]
pressure_rom = pressure_rom[:, :n_times]
t_plot = t_ref[:n_times]

# Plot
t_ms = np.array(t_plot) * 1000
vmax = max(np.abs(pressure_full).max(), np.abs(pressure_rom).max())

fig, axes = plt.subplots(1, 3, figsize=(16, 6))

# Full-order
im0 = axes[0].imshow(pressure_full, aspect='auto', cmap='RdBu_r',
                      extent=[t_ms[0], t_ms[-1], depths[-1], depths[0]],
                      vmin=-vmax, vmax=vmax)
axes[0].axhline(p['sonar_iz'] * p['dz'], color='yellow', linestyle='--', lw=2, label='Source')
axes[0].axhline(0, color='cyan', linestyle='-', lw=2, label='Surface (p=0)')
axes[0].axhline(p['Lz'], color='brown', linestyle='-', lw=2, label='Seafloor')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Depth (m)')
axes[0].set_title(f'Full-order ({full_time:.1f}s)')
axes[0].legend(loc='upper right', fontsize=8)
plt.colorbar(im0, ax=axes[0], label='Pressure (Pa)')

# ROM
im1 = axes[1].imshow(pressure_rom, aspect='auto', cmap='RdBu_r',
                      extent=[t_ms[0], t_ms[-1], depths[-1], depths[0]],
                      vmin=-vmax, vmax=vmax)
axes[1].axhline(p['sonar_iz'] * p['dz'], color='yellow', linestyle='--', lw=2)
axes[1].axhline(0, color='cyan', linestyle='-', lw=2)
axes[1].axhline(p['Lz'], color='brown', linestyle='-', lw=2)
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Depth (m)')
axes[1].set_title(f'ROM q={q} ({rom_time:.2f}s)')
plt.colorbar(im1, ax=axes[1], label='Pressure (Pa)')

# Error
error = pressure_rom - pressure_full
err_max = np.abs(error).max()
im2 = axes[2].imshow(error, aspect='auto', cmap='RdBu_r',
                      extent=[t_ms[0], t_ms[-1], depths[-1], depths[0]],
                      vmin=-err_max, vmax=err_max)
axes[2].set_xlabel('Time (ms)')
axes[2].set_ylabel('Depth (m)')
rel_err = np.linalg.norm(error) / (np.linalg.norm(pressure_full) + 1e-12) * 100
axes[2].set_title(f'Error (rel: {rel_err:.2f}%)')
plt.colorbar(im2, ax=axes[2], label='Error (Pa)')

plt.suptitle('Boundary Condition Verification: Full vs ROM', fontsize=12)
plt.tight_layout()
plt.show()

print(f"\nSpeedup: {full_time/rom_time:.0f}x | Error: {rel_err:.2f}%")
'''

Visualize

## Diagnostics

In [ ]:
'''
# 1. Check that V * V⁻¹ = I
V_inv = np.linalg.inv(V)
identity_error = np.linalg.norm(V @ V_inv - np.eye(n)) / n
print(f"1. V @ V⁻¹ ≈ I? Error: {identity_error:.2e}")

# 2. Check that V⁻¹ @ A @ V = diag(λ)
Lambda_reconstructed = V_inv @ A_dense @ V
off_diag = Lambda_reconstructed - np.diag(np.diag(Lambda_reconstructed))
off_diag_error = np.linalg.norm(off_diag) / np.linalg.norm(Lambda_reconstructed)
print(f"2. V⁻¹AV diagonal? Off-diag error: {off_diag_error:.2e}")

# 3. Check initial conditions match
x0_full = x_start.flatten()
print(f"3. Initial condition:")
print(f"   Full-order x0 norm: {np.linalg.norm(x0_full):.2e}")
print(f"   ROM starts at zero")

# 4. Check time arrays match
print(f"4. Time arrays:")
print(f"   t_rom: {len(t_rom)} points, [{t_rom[0]:.4f}, {t_rom[-1]:.4f}]")
print(f"   t_ref: {len(t_ref)} points, [{t_ref[0]:.4f}, {t_ref[-1]:.4f}]")

# 5. Compare at first few timesteps (Hydrophone 1)
print(f"\n5. First few timesteps (H1):")
for i in range(min(5, len(t_rom))):
    y_full_i = y_ref[0, i]
    y_rom_i = y_rom[i, 0]
    print(f"   t={t_rom[i]*1000:.3f}ms: full={y_full_i:.4e}, rom={y_rom_i:.4e}")

scores = np.abs(b_tilde.flatten()) * np.linalg.norm(c_tilde, axis=0)
freqs_modes = np.abs(eigenvalues.imag) / (2 * np.pi)

# Top 10 modes
idx_top = np.argsort(scores)[::-1][:10]
print("Top 10 modes by I/O coupling:")
print(f"{'Rank':<5} {'Freq (Hz)':<12} {'|b̃|':<12} {'||c̃||':<12} {'Score':<12}")
print("-" * 55)
for rank, idx in enumerate(idx_top):
    b_mag = np.abs(b_tilde[idx, 0])
    c_mag = np.linalg.norm(c_tilde[:, idx])
    print(f"{rank+1:<5} {freqs_modes[idx]:<12.1f} {b_mag:<12.2e} {c_mag:<12.2e} {scores[idx]:<12.2e}")

'''